# Integrating gene regulatory network with multi-omics data
Author: Romana T. Pop^1^

1. Centre for Molecular Medicine Norway (NCMM), Faculty of Medicine, University of Oslo, Oslo, Norway

## Introduction
The emergence of high-throughput omics technologies has resulted in their wide application to cancer studies, greatly increasing our understanding of the disruptions occurring at different molecular levels. The role of gene regulation as a core driver of biological processes and its role in the development and progression of cancer has been well established. Transcriptional regulation, a crucial aspect of gene regulation, can be represented as a network of interactions between regulators (such as transcription factors) and their target genes. These networks are known as gene regulatory networks (GRNs). Several joint dimensionality reduction (JDR) methods and tools for integrating multi-modal data have been developed in recent years. 

In this notebook, we reproduce the analysis presented in *paper link*. We will integrate GRNs with multi-omics data in ten cancer chorts from TCGA and investigte the contribution of GRNs to identifying survival associated factors.

## Necessary data and software
Before starting the analysis, make sure all the necessary data is downloaded and software is installed. For exact reproducibility of the results presented in *paper*, we provide a container with the environment used for the analysis. The data is available on Zenodo at *link*.

If not using the container provided, we recommend cloning this repository and then downloading the data from Zenodo in the cloned repository directory.

In [ ]:
# ensure environment is clean
rm(list=ls())

# install MARMOT
library(devtools)
install_github("rtpop/MARMOT")

# load libraries
library(MARMOT)
library(tidyverse)
library(preprocessCore)
library(RColorBrewer)
library(msigdbr)
library(gridExtra)

## Setting parameters
Here we set some parameters that will be used throughout the notebook.

In [ ]:
# setting working directory
wd <- "results"
setwd(wd)

# specify where plots should be saved
figure_dir <- "figures"

# specify data directories
# this should also have the clinical data so make sure to change the code accordingly
# and to remove these comments once you do so
data_tcga <- "data/TCGA"
data_gep <- "data/GEP"

# specify directory for logs to be saved
log_dir <- "logs"

# specify directory for the benchmarking to be saved
benchmark_dir <- "benchmark"

# defining vector of cancer names for which to do the analysis
# I'm not sure if this is the best way to go about it, but for now...
cancers_tcga <- c("aml", "breast", "colon", "gbm", "kidney", "liver", "lung",
          "melanoma", "ovarian", "sarcoma")
cancers_gep <- "liver"

# defining names for the JDR models that we will run
# also might not be the best way, so review once you have everything in place
model <- c("nonet", "indeg", "out", "both")

# define vector of omic names that will be used
omics_tcga <- c("expression", "methylation", "miRNA", "indegree", "outdegree")
omics_gep <- c("expression", "indegree", "outdegree")

# some intermediate files are provided for ease, set this parameter to FALSE
# if you do not wish to use them and wish to compute them again instead
precomputed <- TRUE

## Preparing the data
**This entire section need not be run if `precomputed = TRUE`**

We reformat the data and prepare it for downstream analysis. The JDR tools used take a list of matrices as input. Here, we create a list of matrices for each cancer type and quantile normalise the indegrees. We also perform PCA on the omics and create a separate list of matrices for the PCA data.

Since we are working with several datasets, the metadata can be messy and inconsistent. Here, we re-format the survival data to ensure the labels are uniform across the cancer types.

See the documentation of [MARMOT](https://github.com/rtpop/MARMOT) for complete details of the formatting and processing done below. 

Since we are applying this processing to many datasets, we first create a wrapper function for processing the omics data and one for processing the survival data. Please note that the paths and filenames below assume the use of the data and directory structure provided on Zenodo. If using this for your own data, you may need to change them accordingly. 

In [ ]:
if (!precomputed) {
    # function for omic processing
    prepare_cancer_data <- function(cancer, data_dir, omic_names, log_dir, wd) {
        print(cancer)
        # Get omic file names
        files <- paste0(data_dir, cancer, "/",
                        c("log_exp", "methy", "log_mirna",
                          "indegree_quant.RData", "outdegree.RData"))
        
        # Quantile normalize indegrees
        indegree_quant_file <- file.path(data_dir, cancer, 
                                         "indegree_quant.RData")
        if (!file.exists(indegree_quant_file)) {
            indegree_file <- file.path(data_dir, cancer, "indegree.RData")
            load(indegree_file)
            indegree <- normalize.quantiles(as.matrix(indegree), copy = FALSE)
            save(indegree, file = indegree_quant_file)
        }

        # Prepare data without PCA
        omics <- prepare_data(omics = files, names = omic_names, pca = FALSE,
                              logs = TRUE,
                              log_name = file.path(log_dir, 
                                                   "prep_data_no_pca_log.txt"))
        save(omics, file = file.path(wd, paste0(cancer, "_omics_no_pca.Rda")))

        # Prepare data with PCA
        omics <- prepare_data(omics = files, names = omic_names, pca = TRUE,
                              logs = TRUE, 
                              log_name = file.path(log_dir, 
                                                   "prep_data_pca_log.txt"),
                              file_name = paste0(cancer, 
                                                 "_omics_pca_results.Rda"))
        save(omics, file = file.path(wd, paste0(cancer, "_omics_pca.Rda")))
    }

    # Function to prepare survival data for a given cancer type
    prepare_survival_data <- function(cancer, data_dir, wd) {
        print(cancer)

        # Define the clinical data path
        clin <- file.path(data_dir, cancer)
        
        # Special handling for 'kidney' cancer type
        if (cancer == "kidney") {
            feature_names <- list(
                sample_id = "submitter_id.samples",
                vital_status = "vital_status.diagnoses",
                time_to_event = c("days_to_death.diagnoses",
                                  "days_to_last_follow_up.diagnoses")
            )
            surv <- prepare_surv(clinical = clin, feature_names = feature_names)
            surv$sample_id <- str_sub(surv$sample_id, end = -2)
        } else {
            feature_names <- list(
                sample_id = "sampleID",
                vital_status = "vital_status",
                time_to_event = c("days_to_death", "days_to_last_followup")
            )
            surv <- prepare_surv(clinical = clin, feature_names = feature_names)
        }

        # Standardize sample IDs
        surv$sample_id <- gsub("-", "\\.", surv$sample_id)

        # Save the survival data
        save(surv, file = file.path(wd, paste0(cancer, "_surv.Rda")))
    }
}

Now we apply it to the TCGA and GEP data.

In [ ]:
if (!precomputed) {
    # Call the preparation function for each cancer type
    # for tcga
    # omics
    lapply(cancers_tcga, prepare_cancer_data, data_dir = data_tcga, 
           omic_names = omics_tcga, log_dir = log_dir, wd = wd)
    # survival       
    lapply(cancers_tcga, prepare_survival_data, data_dir = data_tcga, wd = wd)
       
    
    # for gep
    # omics
    prepare_cancer_data(cancer = cancers_gep, data_dir = data_gep,
                        omic_names = omics_gep, log_dir = log_dir, wd = wd)
    # survival
    prepare_survival_data(cancer = cancers_gep, data_dir = data_gep, wd = wd)
}

# Benchmarking JDR models with and without PCA
One of the challenges of multi-omics integration is the different dimensionalities of the various omics data types which can lead to the results being driven by the difference in dimensionality between omics layers, rather than by biological signal.

Therefore, omics data must generally be filtered prior to JDR. A few filtering strategies are commonly used, such as filtering to the top *n* most variable features or removing non-variable features. However, these approaches may not always lead to an equal proportion of variability being retained from each omics layer.

One approach to filtering in a more data-driven manner is performing principle component analysis (PCA) on the data prior to JDR and using a variance threshold to filter the data. 

Here, we compare the data and models of four JDR tools with and without PCA.

First, we compare the number of features in each omic with and without PCA (reproducing Figures 2 & S2 from the paper).

In [ ]:
# Define vectors for file paths
omics_files_no_pca <- file.path(wd, paste0(cancers, "_omics_no_pca.Rda"))
omics_files_pca <- file.path(wd, paste0(cancers, "_omics_pca.Rda"))
save_paths <- file.path("figures", paste0(cancers, "_data_dim_compare.pdf"))

# Create a list of vectors for omics_files
omics_files <- Map(function(pca, no_pca) c(pca, no_pca), omics_files_pca, omics_files_no_pca)

# Initialize a list to store plots
p_list <- vector("list", length(cancers))
names(p_list) <- cancers

# Use Map for plotting and saving individual plots
p_list <- Map(function(cancer, omics_file, save_path) {
  message("Processing: ", cancer)
  
  # Plot bar plot
  p <- plot_data_dim(data = omics_file, data_labels = c("PCA", "no PCA"),
                     log_x = FALSE, title = cancer, compare = TRUE)
  
  # Save the plot
  ggsave(p, file = save_path, height = 20, width = 20)
  
  return(p)
}, cancers, omics_files, save_paths)

# Combine all plots into a grid
grid_plot <- do.call(grid.arrange, c(grobs = p_list, ncol = 3))
print(grid_plot)
ggsave(grid_plot, file = "figures/data_dim_compare_all_can.pdf", height = 45, width = 40)

Next we benchmark the performance of four JDR tools (MOFA+, JIVE, MCIA and RGCCA) on the data with and without PCA. 

In [ ]:
if (!precomputed) {
  # Ensure the benchmark directory exists
  dir.create(file.path(wd, benchmark_dir), showWarnings = FALSE)

  # Define vectors for omics file paths and save file paths
  omics_files_no_pca <- file.path(wd, paste0(cancers_tcga, "_omics_no_pca.Rda"))
  omics_files_pca <- file.path(wd, paste0(cancers_tcga, "_omics_pca.Rda"))
  save_files_pca <- file.path(wd, benchmark_dir, paste0(cancers_tcga, "_factorisations_pca.Rda"))
  save_files_no_pca <- file.path(wd, benchmark_dir, paste0(cancers_tcga, "_factorisations_no_pca.Rda"))


  # iterate over cancers
  Map(function(cancer, omics_file_pca, omics_file_no_pca, save_file_pca, save_file_no_pca) {
    print(cancer)
    
    # Run factorization
    factorizations_pca <- run_jdr(omic_list = omics_file_pca, seed = 13)
    factorizations_no_pca <- run_jdr(omic_list = omics_file_no_pca, seed = 13)
    
    # Save the factorizations
    save(factorizations_pca, file = save_file_pca)
    save(factorizations_no_pca, file = save_file_no_pca)

  }, cancers_tcga, omics_files_pca, omics_files_no_pca, save_files_pca, save_files_no_pca)
}

We perform univariate cox regression for each factor to asses its association ith survival. The below reproduces Figure S3 from the manuscript.

In [ ]:
# Define vectors for file paths
pca_files <- file.path(benchmark_dir, paste0(cancers_tcga, "_factorisations_pca.Rda"))
no_pca_files <- file.path(benchmark_dir, paste0(cancers_tcga, "_factorisations_no_pca.Rda"))
surv_files <- file.path(wd, paste0(cancers_tcga, "_surv.Rda"))

# Initialize the survival data frame
surv_df <- data.frame()

# process each cancer type
results <- Map(function(cancer, pca_file, no_pca_file, surv_file) {
  message("Processing: ", cancer)

  PCA <- get(load(pca_file))
  noPCA <- get(load(no_pca_file))
  surv <- get(load(surv_file))

  methods <- names(PCA)
  result_list <- list()

  for (method in methods) {
    if (method == "MOFA") {
      pca_fct <- PCA[[method]][[1]]
      nopca_fct <- noPCA[[method]][[1]]
    } else {
      pca_fct <- PCA[[method]]
      nopca_fct <- noPCA[[method]]
    }

    # Run the survival association of the factors
    PCA_cox <- surv_association(pca_fct, surv, univariate = TRUE)
    noPCA_cox <- surv_association(nopca_fct, surv, univariate = TRUE)

    df <- surv_compare(models = list(PCA_cox, noPCA_cox), 
                       model_labels = c("PCA", "no_PCA"), 
                       univariate = TRUE, method = "BH")

    df$cancer <- cancer
    df$method <- method

    result_list[[method]] <- list(df = df, cox_models = list(PCA_cox, noPCA_cox))
  }

  return(result_list)
}, cancers_tcga, pca_files, no_pca_files, surv_files)

# Combine data frames and save the results
for (cancer in cancers_tcga) {
  for (method in names(results[[cancer]])) {
    df <- results[[cancer]][[method]]$df
    surv_df <- rbind(surv_df, df)

    cox_all <- results[[cancer]][[method]]$cox_models
    names(cox_all) <- c("PCA", "no_PCA")

    save(cox_all, file = file.path(benchmark_dir, paste0(cancer, "_cox_models_", method, ".Rda")))
  }
}

save(surv_df, file = file.path(benchmark_dir, "TCGA_surv_all_bench.Rda"))


In [ ]:
# plotting
load("benchmark/TCGA_surv_all_bench.Rda")

method <- unique(surv_df$method)
cols <- palette("Dark2")

for(i in method){
  surv_meth <- surv_df[which(surv_df$method == i),]
  models <- unique(surv_meth$label)
  p <- surv_compare_dotplot(surv_df = surv_meth, models_to_compare = models,
                            colours = c(cols[8], "grey", cols[6]))

  ggsave(p, file=paste0(wd,"/figures/surv_compare_PCA_",i,".pdf"))
}

# Integrating GRNs with multi-omics data using MOFA+
We use MOFA+ to investigate the contribution of GRNs to JDR models and their association with patient survival. We run MOFA+ separately on all omics data types with no GRN information and compared the results with MOFA+ models that included network metrics (indegree, outdegree, and both).

In [ ]:
# running MOFA models
for (cancer in cancers_tcga) {
  load(paste0(wd, "/", cancer, "_omics_pca.Rda"))

  #nonet
  data_nonet <- omics[1:3]
  mofa_nonet <- run_mofa2(data_nonet, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_nonet, file = paste0("MOFA_", cancer, "_pca_nonet.Rda"))

  # with indeg
  data_indeg <- omics[-5]
  mofa_indeg <- run_mofa2(data_indeg, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_indeg, file = paste0("MOFA_", cancer, "_pca_indeg.Rda"))

  # with outdeg
  data_out <- omics[-4]
  mofa_out <- run_mofa2(data_out, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_out, file = paste0("MOFA_", cancer, "_pca_out.Rda"))

  # with both
  data_both <- omics
  mofa_both <- run_mofa2(data_both, n_fct = 5, seed = 13, convergence = "slow", use_basilisk = T)
  save(mofa_both, file = paste0("MOFA_", cancer, "_pca_both.Rda"))
}

As with the benchmarking, we perform univariate cox models for each factor to determine their association with patient survival. We compare the association of the factors to patient survival in the models without GRNs and the models with GRNs, reproducing Figure 3 from the paper. 

In [ ]:
# Define model types
model_types <- c("nonet", "indeg", "out", "both")

# Define file paths for MOFA models and survival data
mofa_files <- lapply(model_types, function(model) {
  file.path(wd, paste0("MOFA_", cancers_tcga, "_pca_", model, ".Rda"))
})
mofa_files <- do.call(c, mofa_files)

surv_files <- file.path(wd, paste0(cancers_tcga, "_surv.Rda"))

# Initialize the survival data frame
surv_df <- data.frame()

# process each cancer for each model type
new_surv_df <- Map(function(cancer, mofa_files, surv_file) {
  message("Processing: ", cancer)

  # Load MOFA models and survival data
  models <- lapply(mofa_files, load)
  models <- lapply(models, get)
  load(surv_file) # loads into 'surv' variable

  # Getting factors
  factors <- lapply(models, function(model) get_factors(model)[[1]])

  # Perform survival associations
  cox_models <- lapply(factors, surv_association, surv = surv, univariate = TRUE)

  # Compare survival models
  df <- surv_compare(models = cox_models, model_labels = model_types,
                     univariate = TRUE, method = "BH")
  df$cancer <- cancer

  # Save individual cox models
  cox_all <- setNames(cox_models, model_types)
  save(cox_all, file = file.path(wd, paste0(cancer, "_cox_models_PCA.Rda")))

  return(df)
}, rep(cancers_tcga, each = length(model_types)), seq(1, length(mofa_files), length(cancers_tcga)), surv_files)

# Combine all results into a single data frame
surv_df <- do.call(rbind, new_surv_df)

# Save the combined survival data frame
save(surv_df, file = paste0(wd, "/TCGA_MOFA_surv_df_all_PCA.Rda"))


In [ ]:
# plotting
if(!exists("surv_df")){
  load(paste0("TCGA_MOFA_surv_df_all_PCA.Rda"))
}

#get models to compare
model_comp <- setdiff(model, "nonet")

#set colours
cols <- palette("Dark2")

for(i in model_comp){
  models_to_compare <- c("nonet", i)
  p <- surv_compare_dotplot(surv_df = surv_df, models_to_compare = models_to_compare, 
                            colours = c(cols[8], "grey", cols[1]))

  ggsave(p, file=paste0(wd, "/figures/surv_compare_",models_to_compare[2],"_bee_5_PCA_quant.png"))
  ggsave(p, file=paste0(wd, "/figures/surv_compare_",models_to_compare[2],"_bee_5_PCA_quant.pdf"))
}

# Investigating survival associated factors in liver cancer
We explore the survival-associated factors identified when including GRN features for the TCGA liver cancer samples. We evaluate which feature types were explained by the survival associated factors and we explore the correlation between the factors without GRNs and those with GRNs to assess whether the GRNs capture different heterogeneity than the other omics, or if they boost a preexisting signal. The below reproduces Figure 4 from the manuscript.

In [ ]:
# Plotting heatmap of the -log10(FDR) of the survival association of the factors (Figure 4A)
load("TCGA_MOFA_surv_df_all_PCA.Rda")
cols <- palette("Dark2")

surv_can <- surv_df %>%
  filter(cancer == "liver")

  p <- .plot_heatmap(surv_can)
  p <- p +
       scale_fill_gradient(low = "white", high = cols[1],
                           limits = c(0,3),
                           breaks = seq(0,3, by = 1)) +
       labs(x = "Model", y = NULL, fill = expression("-log"[10] * "FDR"))
  ggsave(p, file = paste0("figures/liver_fct_sig_heat.pdf"))


In [ ]:
# plotting heatmap of variance explained by each MOFA factor (Figure 4B)
cols <- palette("Dark2")
load("TCGA_MOFA_surv_df_all_PCA.Rda")
load(paste0("MOFA_liver_pca_both.Rda"))

surv_can <- surv_df %>%
  filter(cancer == "liver")

b <- suppressMessages(plot_var_heat(mofa_both))

ggsave(b, file = paste0("figures/",cancer, "_var_explained_both.pdf"),width = 10, height = 9)

In [ ]:
# plotting the correlation of factors from the model without GRNs and the model with GRNs (Figure 4C)
corr_all <- list()
corr_df <- data.frame()

# load models
all <- get(load(paste0("MOFA_liver_pca_both.Rda")))
nonet <- get(load(paste0("MOFA_liver_pca_nonet.Rda")))

# get factors
all_fct <- get_factors(all)[[1]]
nonet_fct <- get_factors(nonet)[[1]]

corr <- fct_corr(all_fct, nonet_fct, labels = c("all", "no GRN"),
                as_data_frame = TRUE, abs = TRUE)

# reformat as data frame
corr_df <- format_fct_corr(corr)

# plot
p <- plot_fct_corr(corr_df, abs = TRUE)

# removing facet headers
p <- p +
  theme(strip.background = element_blank(),  # Remove background of the facet label
        strip.text = element_blank())
ggsave(p, file = paste0("figures/liver_view_exclusion_both_abs.pdf"), width = 10, height = 9)

# save dataframe
save(corr_df, file = "view_exclusion_corr_df_both_abs.Rda")